In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QLineEdit, QVBoxLayout, QPushButton, QTableWidget, QTableWidgetItem, QHBoxLayout
from pymongo import MongoClient, ReturnDocument

class HelloWorldApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('Big Data App')
        layout = QVBoxLayout()

        # Prompt label to enter MongoDB database name
        label_prompt = QLabel('Enter MongoDB database name:', self)
        layout.addWidget(label_prompt)

        # Input field for database name
        db_name_input = QLineEdit(self)
        
        layout.addWidget(db_name_input)

        # Button to check the entered database
        check_button = QPushButton('Check database', self)
        check_button.clicked.connect(lambda: self.check_database(db_name_input.text()))
        # check_button.setStyleSheet("background-color: #6DA9E4;")

        deletedb_button = QPushButton('Delete database', self)
        deletedb_button.clicked.connect(lambda: self.delete_database(db_name_input.text()))

        # Button to check the entered database
        filter1_button = QPushButton('Filter data by score', self)
        filter1_button.clicked.connect(lambda: self.filter_row_by_score(db_name_input.text()))

        # Button to check the entered database
        filter2_button = QPushButton('Filter data by country', self)
        filter2_button.clicked.connect(lambda: self.filter_row_by_country(db_name_input.text()))
        
        hbox1 = QHBoxLayout()
        hbox1.addWidget(check_button)
        hbox1.addWidget(deletedb_button)
        hbox1.addWidget(filter1_button)
        hbox1.addWidget(filter2_button)
        layout.addLayout(hbox1)

        #draw the table on the gui
        self.table_data_widget = QTableWidget(self)
        layout.addWidget(self.table_data_widget)

        # Input field for name to search
        search_input = QLineEdit(self)
        search_input.setPlaceholderText("Enter movie name or movie score")  # Set the placeholder text
        layout.addWidget(search_input)

        # Button to find a row based on name
        self.find_by_name_button = QPushButton('Find by Name', self)
        self.find_by_name_button.clicked.connect(lambda: self.find_row_by_name(search_input.text()))
       
        # Button to find a row based on Score
        self.find_by_score_button = QPushButton('Find by Score', self)
        self.find_by_score_button.clicked.connect(lambda:self.find_row_by_score(search_input.text()))

        self.insert_button = QPushButton('Insert', self)
        self.insert_button.clicked.connect(lambda:self.insert_row(search_input.text()))

        # Button to delete row
        self.delete_button = QPushButton('Delete', self)
        self.delete_button.clicked.connect(lambda:self.delete_row(search_input.text()))

        # Button to update row
        self.update_button = QPushButton('Update', self)
        self.update_button.clicked.connect(lambda:self.update_row(search_input.text()))

        #To put buttons next to each other
        hbox2 = QHBoxLayout()
        hbox2.addWidget(self.find_by_name_button)
        hbox2.addWidget(self.find_by_score_button)
        hbox2.addWidget(self.delete_button)
        hbox2.addWidget(self.update_button)
        hbox2.addWidget(self.insert_button)
        layout.addLayout(hbox2)

        # Label to display the result
        self.result_label = QLabel(self)
        layout.addWidget(self.result_label)

        #draw the table on the gui
        self.table_widget = QTableWidget(self)
        layout.addWidget(self.table_widget)
        
        self.setLayout(layout)
        self.setGeometry(500, 500, 1000, 700)
        self.show()

    def connect_to_database(self):
        try:
            # Connect to MongoDB
            client = MongoClient('mongodb://localhost:27017/')

            # Get the database and collection
            db = client['MoviesDb']
            collection = db['collection']
            self.collection = collection
            return self.collection 
        except Exception as e:
            self.result_label.setText(f"Error connecting to MongoDB: {str(e)}")
    
    def check_database(self, db_name):
        try:
            self.table_data_widget.clear()
            # Connect to MongoDB
            client = MongoClient('mongodb://localhost:27017/')
            # Get the list of available database names
            db_names = client.list_database_names()
            # Check if the entered database name exists in the list
            if db_name in db_names:
                # Get the database and collection
                db = client[db_name]
                #access collection
                collection = db['collection']
                documents = collection.find().limit(5)

                # Display the documents in the first table
                self.table_data_widget.setColumnCount(11)
                self.table_data_widget.setRowCount(5)
                col_headers = []
                row=0
                for document in documents:
                    for col_name in document:
                        if col_name == "crew" or col_name == "overview":
                            continue
                        col_headers.append(col_name)
                    self.table_data_widget.setHorizontalHeaderLabels(col_headers)

                    col=0
                    for col_name in document:
                        if col_name == "crew" or col_name == "overview":
                            continue
                        self.table_data_widget.setItem(row, col, QTableWidgetItem(str(document[col_name])))
                        col+=1
                    row+=1
                self.table_data_widget.resizeColumnsToContents()
                self.result_label.clear()
            else:
                # Display a message if the database is not found
                self.result_label.setText(f"Database '{db_name}' not found.")
                self.table_data_widget.clear()
        except Exception as e:
            # Display an error message if there is a connection issue
            self.result_label.setText(f"Error connecting to MongoDB: {str(e)}")
    
    def delete_database(self, db_name):
        try:
            # Connect to MongoDB
            client = MongoClient('mongodb://localhost:27017/')
            db_names = client.list_database_names()
            #if the database exists then drop it
            if db_name in db_names:
                client.drop_database(db_name)
                self.result_label.setText("Database deleted")
                self.table_data_widget.clear()
            else:
                self.result_label.setText("No database was found")
        except Exception as e:
            self.result_label.setText(f"Error connecting to MongoDB: {str(e)}")

    def insert_row(self, name):
        collection = self.connect_to_database()
        result = collection.insert_one({'names':name})
        if result:
            self.result_label.setText("Record added")
        else:
            self.result_label.setText("Record could not be added")

    def filter_row_by_country(self, db_name):
        try:
            collection = self.connect_to_database()
            #display movies with a filter score  of 75
            documents = collection.find({'country': 'US'}).limit(5)
            
            self.table_data_widget.setColumnCount(11)
            self.table_data_widget.setRowCount(5)
            col_headers = []
            row=0
            for document in documents:
                for col_name in document:
                    if col_name == "crew" or col_name == "overview":
                        continue
                    col_headers.append(col_name)
                self.table_data_widget.setHorizontalHeaderLabels(col_headers)
                col=0
                for col_name in document:
                    if col_name == "crew" or col_name == "overview":
                        continue
                    self.table_data_widget.setItem(row, col, QTableWidgetItem(str(document[col_name])))
                    col+=1
                row+=1
            self.table_data_widget.resizeColumnsToContents()
        except Exception as e:
            # Display an error message if there is a connection issue
            self.result_label.setText(f"Error connecting to MongoDB: {str(e)}")

    def delete_row(self, name):
        collection = self.connect_to_database()
        # Find the row/document based on the name and then update the name to
        result = collection.delete_one({'names': name})
        if result.deleted_count == 1:
            # Display a message if no row/document is found
            self.result_label.setText("Record deleted")
            self.table_widget.clear()
        else:
            # Display a message if no row/document is found
            self.result_label.setText(f"No row found with the name '{name}'.")
    
    def filter_row_by_score(self, db_name):
        try:
            collection = self.connect_to_database()
            #display movies with a filter score  of 75
            documents = collection.find({'score': '75.0'}).limit(5)
            
            self.table_data_widget.setColumnCount(11)
            self.table_data_widget.setRowCount(5)
            col_headers = []
            row=0
            for document in documents:
                for col_name in document:
                    if col_name == "crew" or col_name == "overview":
                        continue
                    col_headers.append(col_name)
                self.table_data_widget.setHorizontalHeaderLabels(col_headers)
                col=0
                for col_name in document:
                    if col_name == "crew" or col_name == "overview":
                        continue
                    self.table_data_widget.setItem(row, col, QTableWidgetItem(str(document[col_name])))
                    col+=1
                row+=1
            self.table_data_widget.resizeColumnsToContents()
        except Exception as e:
            # Display an error message if there is a connection issue
            self.result_label.setText(f"Error connecting to MongoDB: {str(e)}")
        
    def update_row(self, name):       
        collection = self.connect_to_database()
        # Find the row/document based on the name 
        # and then update the name and return document is set to after to change it inplace
        result = collection.find_one_and_update({'names': name},{'$set':{'names':"Name_updated"}},return_document=ReturnDocument.AFTER)
        if result:
            # Display the found row/document in the table
            self.populate_table(result)
        else:
            # Display a message if no row/document is found
            self.result_label.setText(f"No row found with the name '{name}'.")

    def find_row_by_score(self, score):
        """function to return row by score but must be in double"""
        collection = self.connect_to_database()
        
        # Find the row/document based on the score
        result = collection.find_one({'score': score})
        if result:
            self.populate_table(result) 
        else:
            # Display a message if no row/document is found
            self.result_label.setText(f"No row found with the score '{score}'.")

    def find_row_by_name(self, name):
        """function to return row by name but it is case sensitive"""
        collection = self.connect_to_database()

        # Find the row/document based on the name
        result = collection.find_one({'names': name})
        if result:
            # Display the found row/document in the table
            self.populate_table(result)
        else:
            # Display a message if no row/document is found
            self.result_label.setText(f"No row found with the name '{name}'.")

    def populate_table(self, data):
        #Set the table headers
        col_header=[]
        for col_name in data:
            if col_name == "crew" or col_name == "overview":
                continue
            col_header.append(col_name)

        #Set table column headers
        self.table_widget.setColumnCount(len(col_header))
        self.table_widget.setHorizontalHeaderLabels(col_header)
        self.table_widget.setRowCount(1)

        #set row values
        col = 0
        for col_name in data:
            if col_name == "crew" or col_name == "overview":
                continue
            self.table_widget.setItem(0, col, QTableWidgetItem(str(data[col_name]))) #row 0 col 0
            col+=1
        self.table_widget.resizeColumnsToContents()

In [2]:
if __name__ == '__main__':
    app = QApplication(sys.argv)
    hello_world_app = HelloWorldApp()
    sys.exit(app.exec_())